# Exploração Numérica do Problema

Sumário  
Definição do Problema  
        ---A Classificação  
        ---Os atributos  
Exploração Numérica  
        ---Média, mediana e percentis  
        ---Média, mediana e percentis por classe  
        --- Gráficos  
Árvore de Decisão  
        ---Completa  
        ---Reduzida  

In [1]:
import pandas
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import preprocessing
import sklearn as skl

import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from statistics import mean
from numpy.linalg import norm
from matplotlib.colors import ListedColormap
import random

In [2]:
#importação da base de dados
file = "covtype.data"

#nome dos atributos
#estamos descartando os atributos que descrevem o tipo do solo
names = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area_Rawah', 'Wilderness_Area_Neota', 'Wilderness_Area_Comanche', 'Wilderness_Area_Cache', 'Cover_Type']

#de 0 a 13 são os atributos listados acima, e 54 é a classificação
usecols = list(range(0, 14)) + [54]

#especifico o tipo de alguns parametros(os que não são simplesmente numéricos)
dtype = {'Cover_Type': 'category', 'Wilderness_Area_Rawah' : bool, 'Wilderness_Area_Neota' : bool, 'Wilderness_Area_Comanche' : bool, 'Wilderness_Area_Cache' : bool}

#lê o arquivo num pandas.dataframe
dataset = pandas.read_csv(file, header = None, usecols = usecols, names = names, dtype = dtype)

#adicionando uma coluna adicional para sintetizar os 4 boleanos que representam a Wilderness_area. 
#para uma única instância, somente um dos 4 booleanos pode ser verdadeiro, logo eles, em realidade, funcionam como uma categorização
new_column = pandas.Series([1 if dataset['Wilderness_Area_Rawah'][i] else 
                            2 if dataset['Wilderness_Area_Neota'][i] else
                            3 if dataset['Wilderness_Area_Comanche'][i] else
                            4 for i in range(len(dataset.index)) ], dtype="category")
#elimina as colunas reduzidas
dataset = dataset.drop(columns=['Wilderness_Area_Rawah', 'Wilderness_Area_Neota', 'Wilderness_Area_Comanche', 'Wilderness_Area_Cache'])
#insere nova coluna na posição 10
dataset.insert(loc = 10, column = 'Wilderness_Area', value = new_column)

#atualiza names para refletir a mudança acima
names = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area', 'Cover_Type']

print("Dataframe com %d exemplares e %d atributos importado com sucesso" % (dataset.values.shape[0], dataset.values.shape[1]))

Dataframe com 581012 exemplares e 12 atributos importado com sucesso


In [ ]:
#corr = dataset.corr()['Cover_Type'][dataset.corr()['Cover_Type'] < 1].abs()
#corr.sort(ascending = False)
#corr.head()

In [ ]:
#atributos importados e seus tipos de dados
print(dataset.dtypes)

In [ ]:
#quantidade de exemplares por classificação
dataset.groupby('Cover_Type').size()

In [ ]:
#um exemplar da base de dados, para visualização
dataset.head(1)

In [ ]:
#sumário dos dados, geral
dataset.describe()

In [ ]:
#sumário dos dados, por classificação
gp = dataset.groupby('Cover_Type')
for name in names:
    print(name)
    display(gp[name].describe())

In [ ]:
#faz um histograma do atributo:
def histogram(att_name):
    
    fig, ax = plt.subplots()
    dataset[att_name].hist()
    ax.set(xlabel=att_name, ylabel = 'Number of Samples', title=att_name + ' Histogram')
    plt.show()

In [ ]:
histogram('Elevation')

In [ ]:
#um histograma da distância até foco de incêndio, que se provou um importante atributo
histogram('Horizontal_Distance_To_Fire_Points')

In [ ]:
#um histograma da distância até rodovias, que se provou um importante atributo
histogram('Horizontal_Distance_To_Roadways')

In [ ]:
#um histograma distribuição entre as categorias de cobertura
histogram('Cover_Type')

In [ ]:
#um histograma distribuição do sombreamento ao meio dia, que não ajuda muito
histogram('Hillshade_Noon')

In [ ]:

#faz um scatter plot maneiro
seed = 7
N = 250
#markerTypes = {'1':'.', '2': 'x', '3': 'o', '4': '^', '5': '1', '6': '2', '7': '3'}
markerTypes = {'1':'.', '2': '.', '3': '.', '4': '.', '5': '.', '6': '.', '7': '.'}
points = []
markers = []
gp = dataset.groupby('Cover_Type', sort = False)
samples_stratified = gp.apply(lambda x: x.sample(n = N, random_state = seed))
#para cada classificação, seleciona N amostras aleatórias pra serem plotadas

gp_random = dataset
samples_random = gp_random.apply(lambda x: x.sample(n = 7*N, random_state = seed))

#plota grafico name1 x name2
def scatter_plot(name1, name2, stratified = True):
    title_name = name1 + ' x ' + name2 + (' (E)' if stratified else ' (A)')
    samples = samples_stratified if stratified else samples_random
    fig, ax = plt.subplots()
    for cover_type, group in samples.groupby('Cover_Type'):

        plt.scatter(group[name1], group[name2], marker = markerTypes[cover_type])

    ax.set(xlabel=name1, ylabel=name2,
           title=title_name)
    plt.show()

In [ ]:
#plota grafico Elevation x Slope
scatter_plot('Elevation', 'Slope', stratified = True)


In [ ]:
#plota grafico Elevation x Slope
scatter_plot('Elevation', 'Slope', stratified = False)

In [ ]:
#plota grafico Elevation x Slope
scatter_plot('Elevation', 'Horizontal_Distance_To_Roadways', stratified = True)

#plota grafico Elevation x Slope
scatter_plot('Elevation', 'Horizontal_Distance_To_Roadways', stratified = False)


In [ ]:
#plota grafico Elevation x Slope
scatter_plot('Elevation', 'Horizontal_Distance_To_Fire_Points', stratified = True)

#plota grafico Elevation x Slope
scatter_plot('Elevation', 'Horizontal_Distance_To_Fire_Points', stratified = False)

In [ ]:
#plota grafico Elevation x Slope
scatter_plot('Horizontal_Distance_To_Roadways', 'Horizontal_Distance_To_Fire_Points', stratified = True)

#plota grafico Elevation x Slope
scatter_plot('Horizontal_Distance_To_Roadways', 'Horizontal_Distance_To_Fire_Points', stratified = False)

In [ ]:
#plota grafico Elevation x Slope
scatter_plot('Elevation', 'Elevation', stratified = True)

#plota grafico Elevation x Slope
scatter_plot('Elevation', 'Elevation', stratified = False)

In [ ]:
#plota grafico Elevation x Slope
scatter_plot('Hillshade_3pm', 'Hillshade_9am', stratified = True)

#plota grafico Elevation x Slope
scatter_plot('Hillshade_3pm', 'Hillshade_9am', stratified = False)

In [ ]:
#plota grafico Elevation x Slope
scatter_plot('Hillshade_Noon', 'Slope', stratified = True)

#plota grafico Elevation x Slope
scatter_plot('Hillshade_Noon', 'Slope', stratified = False)

In [ ]:
#plota grafico Elevation x Slope
scatter_plot('Hillshade_Noon', 'Aspect', stratified = True)

#plota grafico Elevation x Slope
scatter_plot('Hillshade_3pm', 'Aspect', stratified = False)

# Árvore de Decisão

In [25]:
#algumas funções auxiliares
def print_feature_importances(dtc):
    fi = dtc.feature_importances_
    for i in range(len(fi)):
        msg = "%s: %f" % (names[i], fi[i])
        print(msg)
       
def print_decision_tree_info(estimator, verbose = False):
    
    print(type(estimator))
    n_nodes = estimator.tree_.node_count
    children_left = estimator.tree_.children_left
    children_right = estimator.tree_.children_right
    feature = estimator.tree_.feature
    threshold = estimator.tree_.threshold

    node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
    is_leaves = np.zeros(shape=n_nodes, dtype=bool)
    stack = [(0, -1)]  # seed is the root node id and its parent depth
    while len(stack) > 0:
        node_id, parent_depth = stack.pop()
        node_depth[node_id] = parent_depth + 1

        # If we have a test node
        if (children_left[node_id] != children_right[node_id]):
            stack.append((children_left[node_id], parent_depth + 1))
            stack.append((children_right[node_id], parent_depth + 1))
        else:
            is_leaves[node_id] = True

    if(verbose): print("The binary tree structure has %s nodes and has "
          "the following tree structure:"
          % n_nodes)
    for i in range(n_nodes):
        if is_leaves[i]:
            if(verbose): print("%snode=%s leaf node." % (node_depth[i] * "\t", i))
        else:
            if (verbose):print("%snode=%s test node: go to node %s if X[:, %s] <= %s else to "
                  "node %s."
                  % (node_depth[i] * "\t",
                     i,
                     children_left[i],
                     feature[i],
                     threshold[i],
                     children_right[i],
                     ))
    print()
    
    print("Node count:", n_nodes)
    print("Leaf node count:", sum(b for b in is_leaves))
    print("Max node depth:", node_depth.max())
    
def get_decision_tree_info(estimator, verbose = False):
    
    n_nodes = estimator.tree_.node_count
    children_left = estimator.tree_.children_left
    children_right = estimator.tree_.children_right
    feature = estimator.tree_.feature
    threshold = estimator.tree_.threshold

    node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
    is_leaves = np.zeros(shape=n_nodes, dtype=bool)
    stack = [(0, -1)]  # seed is the root node id and its parent depth
    while len(stack) > 0:
        node_id, parent_depth = stack.pop()
        node_depth[node_id] = parent_depth + 1

        # If we have a test node
        if (children_left[node_id] != children_right[node_id]):
            stack.append((children_left[node_id], parent_depth + 1))
            stack.append((children_right[node_id], parent_depth + 1))
        else:
            is_leaves[node_id] = True

    if(verbose): print("The binary tree structure has %s nodes and has "
          "the following tree structure:"
          % n_nodes)
    for i in range(n_nodes):
        if is_leaves[i]:
            if(verbose): print("%snode=%s leaf node." % (node_depth[i] * "\t", i))
        else:
            if (verbose):print("%snode=%s test node: go to node %s if X[:, %s] <= %s else to "
                  "node %s."
                  % (node_depth[i] * "\t",
                     i,
                     children_left[i],
                     feature[i],
                     threshold[i],
                     children_right[i],
                     ))
    print()
    
    return (n_nodes, sum(b for b in is_leaves), node_depth.max())

In [4]:
array = dataset.values
X = array[:, 0:11]
Y = array[:, 11]

#preprocessa dataset
#estava preprocessando, mas não fazia diferença
#X_scale_temp = preprocessing.scale(X[:, 0:10])
#X_scaled = np.append(X_scale_temp, X[:, 10].reshape(-1, 1), axis = 1)


validation_size = 0.2
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size = validation_size, random_state = seed)

In [34]:
scoring = 'accuracy'
seed = 7
def EstimarAcuraciaMetodo(metodo, k = 10):
    kfold = model_selection.KFold(n_splits = k, shuffle = True, random_state = 121)#, random_state = seed)
    cv_results = model_selection.cross_validate(metodo, X_train, Y_train, cv = kfold, scoring = scoring)
    print(cv_results)
    msg = "%s média entre os %d folds: %f, com desvio padrão de %f ,em " % (scoring, k, cv_results['test_score'].mean(), cv_results['test_score'].std())
    print(msg)
    msg2 = "tempo médio de treino: %f, para %d samples por fit" % (cv_results['fit_time'].mean(), (len(X_train) // k) * (k-1))
    print(msg2)

def EstimarAcuraciaMetodo_dtc(metodo, k = 10):
    kfold = model_selection.KFold(n_splits = k, shuffle = True, random_state = 121)#, random_state = seed)
    cv_results = model_selection.cross_validate(metodo, X_train, Y_train, cv = kfold, scoring = scoring, return_estimator=True)
    #print(cv_results)
    msg = "%s média entre os %d folds: %f, com desvio padrão de %f ,em " % (scoring, k, cv_results['test_score'].mean(), cv_results['test_score'].std())
    print(msg)
    msg2 = "tempo médio de treino: %f, para %d samples por fit" % (cv_results['fit_time'].mean(), (len(X_train) // k) * (k-1))
    print(msg2)
    nodes_n = []
    for dtc in cv_results['estimator']:
        nodes_n.append(get_decision_tree_info(dtc)[0])
    print("Média de %f nós por árvore gerada" % mean(nodes_n))
    print(nodes_n)

In [35]:
#tentar descobrir o nivel de importancia dos atributos baseado na arvore de decisão
dtc = DecisionTreeClassifier(random_state = seed)
dtc.fit(X_train, Y_train)
dtc.score(X_validation, Y_validation)

0.9265681608908548

In [36]:
#feature importance diz a importância relativa de cada atributo
print_feature_importances(dtc)

Elevation: 0.346055
Aspect: 0.031844
Slope: 0.022997
Horizontal_Distance_To_Hydrology: 0.070997
Vertical_Distance_To_Hydrology: 0.056597
Horizontal_Distance_To_Roadways: 0.164803
Hillshade_9am: 0.034835
Hillshade_Noon: 0.039353
Hillshade_3pm: 0.027806
Horizontal_Distance_To_Fire_Points: 0.165863
Wilderness_Area: 0.038850


In [37]:
print_decision_tree_info(dtc)

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

Node count: 56881
Leaf node count: 28441
Max node depth: 41


In [38]:
dtcN = DecisionTreeClassifier(random_state = seed)
EstimarAcuraciaMetodo_dtc(dtcN)

accuracy média entre os 10 folds: 0.922661, com desvio padrão de 0.001399 ,em 
tempo médio de treino: 6.071490, para 418320 samples por fit
<class 'sklearn.tree.tree.DecisionTreeClassifier'>

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

Média de 52896.000000 nós por árvore gerada
[52857, 53283, 53079, 53393, 52847, 52277, 52979, 52787, 52531, 52927]


In [39]:
#experimetando com profundidades máximas. Já fiz a sem limite de profundidade aqui em cima
n_list = [1, 3, 5, 10,20, 30]
for i in n_list:
    print("Arvore de decisão limitada a profundidade de %d" % i)
    dtc1 = DecisionTreeClassifier(random_state = seed, max_depth = i)
    EstimarAcuraciaMetodo_dtc(dtc1)
    print(" ")

Arvore de decisão limitada a profundidade de 1
accuracy média entre os 10 folds: 0.633617, com desvio padrão de 0.001610 ,em 
tempo médio de treino: 1.220608, para 418320 samples por fit
<class 'sklearn.tree.tree.DecisionTreeClassifier'>

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

Média de 3.000000 nós por árvore gerada
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
 
Arvore de decisão limitada a profundidade de 3
accuracy média entre os 10 folds: 0.674546, com desvio padrão de 0.001216 ,em 
tempo médio de treino: 1.814702, para 418320 samples por fit
<class 'sklearn.tree.tree.Decisio

In [40]:
#novamente árvore de decisão, porém reduzida a tres attributos
X_3 = np.zeros((array.shape[0], 3))
X_3[:, 0] = dataset['Elevation']
X_3[:, 1] = dataset['Horizontal_Distance_To_Roadways']
X_3[:, 2] = dataset['Horizontal_Distance_To_Fire_Points']
Y = dataset.values[:, 11]
validation_size = 0.2
seed = 7
X_train_3, X_validation_3, Y_train_3, Y_validation_3 = model_selection.train_test_split(X_3, Y, test_size = validation_size, random_state = seed)


In [44]:
def EstimarAcuraciaMetodo_dtc_3(metodo, k = 10):
    kfold = model_selection.KFold(n_splits = k, shuffle = True, random_state = 121)#, random_state = seed)
    cv_results = model_selection.cross_validate(metodo, X_train_3, Y_train_3, cv = kfold, scoring = scoring, return_estimator=True)
    #print(cv_results)
    msg = "%s média entre os %d folds: %f, com desvio padrão de %f ,em " % (scoring, k, cv_results['test_score'].mean(), cv_results['test_score'].std())
    print(msg)
    msg2 = "tempo médio de treino: %f, para %d samples por fit" % (cv_results['fit_time'].mean(), (len(X_train_3) // k) * (k-1))
    print(msg2)
    nodes_n = []
    for dtc in cv_results['estimator']:
        nodes_n.append(get_decision_tree_info(dtc)[0])
    print("Média de %f nós por árvore gerada" % mean(nodes_n))
    print(nodes_n)

In [41]:
dtc_3 = DecisionTreeClassifier(random_state = seed)
dtc_3.fit(X_train_3, Y_train_3)
dtc_3.score(X_validation_3, Y_validation_3)

0.8172766623925372

In [42]:
print_feature_importances(dtc_3)

Elevation: 0.432133
Aspect: 0.276885
Slope: 0.290982


In [43]:
print_decision_tree_info(dtc_3)

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

Node count: 136457
Leaf node count: 68229
Max node depth: 47


In [45]:
dtc_3_N = DecisionTreeClassifier(random_state = seed)
EstimarAcuraciaMetodo_dtc_3(dtc_3_N)

accuracy média entre os 10 folds: 0.815591, com desvio padrão de 0.001762 ,em 
tempo médio de treino: 2.616317, para 418320 samples por fit
<class 'sklearn.tree.tree.DecisionTreeClassifier'>

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

<class 'sklearn.tree.tree.DecisionTreeClassifier'>

Média de 124847.200000 nós por árvore gerada
[124833, 125285, 124607, 124595, 124583, 124985, 124883, 124831, 125111, 124759]


# KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 11)
knn.fit(X_train, Y_train)
knn.score(X_validation, Y_validation)

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 11)
knn.fit(X_train_3, Y_train_3)
knn.score(X_validation_3, Y_validation_3)